# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [14]:
def priest(sex, age, respRate, oxySat, heartRate, sysBP, temp, alertness, inspOyg, perform):
    """
    (String, integer, integer, integer, integer, integer, float, String, String, String) -> float)

    Priest function calculates risk percentage like Predicts adverse outcome in adults presenting to ED with suspected COVID-19.

    Priest function expects sex, alertness, inspOxy and perform in String and remaining all are in float.

    Priest function calculates risk based on the algorithm provided in PRIEST mdcalc

    >>> priest("Male", 18, 12, 0.96, 115, 115, 38.1, "ALERT", "AIR", "unrestricted normal activity")
    0.03

    >>> priest("FEMALE", 18, 24, 0.96, 41, 115, 35.1, "confused or not alert", "AIR", "unrestricted normal activity")
    0.18

    >>> priest("FEMALE", 40, 8, 0.89, 105, 101, 38.1, "ALERT", "supplemental oxygen", "unrestricted normal activity")
    0.34

    """

    sex = sex.lower()
    score = 0

    if sex=="male":
        score += 1
    if age > 80:
        score = score + 4
    elif age >= 50 and age<=65:
        score = score + 2
    elif age >= 66 and age<=80:
        score = score + 3
    if respRate < 9:
        score = score +  3
    elif respRate >= 9 and respRate <= 11:
        score = score +  1
    elif respRate >= 21 and respRate <= 24:
        score = score +  2
    elif respRate > 24:
        score = score +  3
    if oxySat >= 0.94 and oxySat <= 0.95:
        score = score +  1
    elif oxySat >= 0.92 and oxySat <= 0.93:
        score = score +  2
    elif oxySat < 0.92:
        score = score +  3
    if heartRate < 41:
        score = score +  3
    elif heartRate >= 41 and heartRate <= 50:
        score = score +  1
    elif heartRate >= 91 and heartRate <= 110:
        score = score +  1
    elif heartRate >= 111 and heartRate <= 130:
        score = score +  2
    elif heartRate > 130:
        score = score +  3
    if sysBP < 91:
        score = score +  3
    elif sysBP >= 91 and sysBP <= 100:
        score = score +  2
    elif sysBP >= 101 and sysBP <= 110:
        score = score +  1
    elif sysBP > 219:
        score = score +  3
    if temp < 35.1:
        score = score +  3
    elif temp >= 35.1 and temp <= 36.0:
        score = score +  1
    elif temp >= 38.1 and temp <= 39.0:
        score = score +  1
    elif temp > 39.0:
        score = score +  2
    alertness = alertness.lower()
    if alertness == "confused or not alert":
        score = score +  3
    inspOyg = inspOyg.lower()
    if inspOyg == "supplemental oxygen":
        score = score +  2
    perform = perform.lower()
    if perform == "limited strenuous activity, can do light activity":
        score = score +  1
    elif perform == "limited activity, can self-care":
        score = score +  2
    elif perform == "limited self-care":
        score = score +  3
    elif perform == "bed/chair bound, no self-care":
        score = score +  4

    rPer = [0.01, 0.01, 0.02, 0.02, 0.03, 0.09, 0.15, 0.18, 0.22, 0.26, 0.29, 0.34, 0.38, 0.46, 0.47, 0.49, 0.55, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,0.99]
    if score>=26:
        return 0.99
    return rPer[score]

In [15]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest("Male", 18, 12, 0.96, 115, 115, 38.1, "ALERT", "AIR", "unrestricted normal activity")
Expecting:
    0.03
ok
Trying:
    priest("FEMALE", 18, 24, 0.96, 41, 115, 35.1, "confused or not alert", "AIR", "unrestricted normal activity")
Expecting:
    0.18
ok
Trying:
    priest("FEMALE", 40, 8, 0.89, 105, 101, 38.1, "ALERT", "supplemental oxygen", "unrestricted normal activity")
Expecting:
    0.34
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [16]:
import json
import csv
import requests

def find_hospital(age, sex, rper):
    """
    (integer, String, float) -> String

    find_hospital function find the hospital based on age, sex and risk percentage.

    find_hospital function expects age, risk percentage in float and sex in String and produces hospital name in String

    find_hospital function find hospital on API by passing age, sex and rper as query parameters


    >>> find_hospital(40, "female", 0.34)
    'Emory Dunwoody Medical Center'

    >>> find_hospital(40, "female", 0.03)
    'Select Specialty Hospital - Northeast Atlanta'

    >>> find_hospital(18, "MALE", 0.26)
    'Southwest Hospital and Medical Center'

    """
    sex = sex.lower()
    httpurll = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"
    outPut = requests.get(httpurll, params={"age": age, "sex": sex, "risk_pct": rper})

    if outPut.status_code==200:
        outPutJSON = outPut.json()
        return outPutJSON['hospital']
    else:
        print("URL is not working and status code is : " + outPut.status_code)

In [17]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, "female", 0.34)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(40, "female", 0.03)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(18, "MALE", 0.26)
Expecting:
    'Southwest Hospital and Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [18]:
import json
import csv
import requests

def get_address(hospName):
    """
    (String) -> String

    get_address function find the address of hospital based on hospital name from the url provided

    get_address function expects hospital name as String and return address in String

    get_address function find address by use API, where it has hospital name json data in it.

    >>> get_address("Select Specialty Hospital - Northeast Atlanta")
    '1821 CLIFTON ROAD NE'
    >>> get_address("Hughes Spalding Childrens Hospital")
    '1711 TULLIE CIRCLE NE'
    >>> get_address("Southwest Hospital and Medical Center")
    '501 FAIRBURN ROAD SW'
    """

    httpurll = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"
    outPut = requests.get(httpurll)
    hospName=hospName.upper()
    if outPut.status_code==200:
        outPutJSON = outPut.json()
        if hospName in outPutJSON:
            return outPutJSON[hospName]["ADDRESS"]
        else:
            return " "
    else:
        print("URL is not working and status code is : " + outPut.status_code)

## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [19]:
import json
import csv
import requests

def process_people(floc):
    """
    (String) -> Dictionary

    process_people function finds the risk percentage, hospital to visit and hospital address
    from group of patients.

    process_people function accepts floc as String and return Dictionary in json format

    It gathers data in the csv file from the API and computes risk percentage, hospital to visit
    and address of the hospital and append each patient with information including all factors and risk percentage,
    hospital name and address to the dictionary in json format.

    """

    appendJSON = {}
    outPut = requests.get(floc)
    if outPut.status_code==200:
       fileData = outPut.text.split("\n")
       fileDataWithoutDelimiter = csv.reader(fileData, delimiter='|')
       patient, sex, age, respRate, oxySat, heartRate, sysBP, temp, alertness, inspOyg, perform = next(fileDataWithoutDelimiter)
       for x in fileDataWithoutDelimiter:
           if not any(field != '' for field in x):
               break
           patient, sex, age, respRate, oxySat, heartRate, sysBP, temp, alertness, inspOyg, perform = x
           rPer = priest(sex, float(age), float(respRate), float(oxySat), float(heartRate), float(sysBP), float(temp), alertness, inspOyg, perform)
           hospName = find_hospital(int(age), sex, rPer)
           adds = get_address(hospName)
           df = [sex, float(age), float(respRate), float(oxySat), float(heartRate), float(sysBP), float(temp), alertness, inspOyg, perform, rPer, hospName, adds]
           appendJSON[patient] = df

       return appendJSON


    else:
        print("URL is not working and status code is : " + outPut.status_code)

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [20]:
import json
import csv
import requests

"""
It compares the actual data from API with the processed data
from above methods and displays whether both are equal or not
"""

httpurll = "https://hds5210-data.s3.amazonaws.com/people_results.json"

outPut = requests.get(httpurll)
actualData = outPut.json()
computedData = process_people("https://hds5210-data.s3.amazonaws.com/people.psv")
if json.loads(json.dumps(actualData)) == json.loads(json.dumps(computedData)):
    print("Actual JSON Data and Computed JSON Data are equal")
else:
    print("Both Are Not Equal")

Actual JSON Data and Computed JSON Data are equal


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---